## Execute a Classical Optimization model using Airline Crew Pairing Algorithm from AWS Marketplace 


The solution uses a classical optimization based approach to derive pairings, i.e., combination of flight legs. 



This sample notebook shows you how to execute a Classical Optimization model using Airline Crew Pairing Algorithm from AWS Marketplace 

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to For Seller to update: Airline Crew Pairing Optimization. 

#### Contents:
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure dataset](#B.-Configure-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Execute optimization model](#3.-Execute-optimization-model)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Execute model](#3.2-Execute-model)
    1. [Visualize Output](#3.3-Inspect-the-Output-in-S3)
1. [Clean-up](#4.-Clean-up)
	1. [Unsubscribe to the listing (optional)](#Unsubscribe-to-the-listing-(optional))


#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page Airline Crew Pairing Optimization
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [13]:
algo_arn ='arn:aws:sagemaker:us-east-2:786796469737:algorithm/crew-pairing-classical-sagemaker-listing-test-output-path-2'

### 2. Prepare dataset

In [14]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from urllib.parse import urlparse
import boto3
import urllib.request
import numpy as np
import tarfile
from zipfile import ZipFile
import pandas as pd
from pprint import pprint

#### A. Dataset format expected by the algorithm

The algorithm requires data in the format as described for best results:
* Input File name should be data.zip
* Within the zip file, inputs must be provided as a CSV file with mandatory information in columns.
* The input data files must contain all columns specified in input data description; other columns will be ignored.
* For detailed instructions, please refer sample notebook and algorithm input details

#### B. Configure dataset

In [15]:
training_dataset='Input/data.zip'

#### C. Upload datasets to Amazon S3

In [16]:
sagemaker_session = sage.Session()
bucket=sagemaker_session.default_bucket()

In [17]:
# training input location
common_prefix = "crew_pairing"
training_input_prefix = common_prefix + "/training-input-data"
TRAINING_WORKDIR = "Input"
training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix)
print("Training input uploaded to " + training_input)

Training input uploaded to s3://sagemaker-us-east-2-786796469737/crew_pairing/training-input-data


## 3. Execute optimization model

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to execute a optimization model. 

### 3.1 Set up environment

In [18]:
role = get_execution_role()

In [19]:
output_location = 's3://{}/crew_pairing/{}'.format(bucket, 'output')

### 3.2 Execute model

For information on creating an `Estimator` object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [ ]:
training_instance_type='ml.m5.4xlarge'

In [21]:
#Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="crew-pairing-training",
    role=role,
    train_instance_count=1,
    train_instance_type=training_instance_type,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type=training_instance_type
)
#Run the training job.
estimator.fit({"training": training_input})

2022-07-04 12:21:13 Starting - Starting the training job...
2022-07-04 12:21:29 Starting - Preparing the instances for trainingProfilerReport-1656937273: InProgress
......
2022-07-04 12:22:43 Downloading - Downloading input data...
2022-07-04 12:22:59 Training - Training image download completed. Training in progress.Starting the training.
Time taken for solver: 54.604912519454956
1
{'pairings': 248, 'min flying hr time': 1.3, 'max flying hr time': 28.58, 'avg flying hr time': 7.59, 'flying hours standard deviation ': 7.51, 'number of deadhead flights': 1, 'number of flight legs not covered': 0, 'min rest time': 18.0, 'max rest time': 872.0}
Success

2022-07-04 12:35:42 Uploading - Uploading generated training model
2022-07-04 12:35:42 Completed - Training job completed
Training seconds: 779
Billable seconds: 779


See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

In [23]:
#output is available on following path
estimator.output_path

's3://sagemaker-us-east-2-786796469737/crew_pairing/output'

## Note: Inferencing is done within training pipeline. Real time inference endpoint/batch transform job is not required.

### 3.3 Inspect the Output in S3

In [31]:
from urllib.parse import urlparse

parsed_url = urlparse(estimator.output_path)
bucket_name = parsed_url.netloc
file_key = parsed_url.path[1:]+'/'+estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

s3_client = sagemaker_session.boto_session.client('s3')

response = s3_client.get_object(Bucket = sagemaker_session.default_bucket(), Key = file_key)

In [38]:
bucketFolder = estimator.output_path.rsplit('/')[3] +'/output/'+estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

In [41]:
import boto3
s3_conn = boto3.client("s3")
bucket_name=bucket
with open('output.tar.gz', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder, f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [43]:
with tarfile.open('output.tar.gz') as file:
    file.extractall('./output')
with ZipFile('./output/result.zip', "r") as output_zip:
    res = json.loads(output_zip.open('results.json').read().decode('utf-8'))
    kpi = json.loads(output_zip.open('kpi.json').read().decode('utf-8'))

In [53]:
print('Crew Pairings:')
pprint(res)

Crew Pairings:
{'pairing 1': ['LEG_08_26',
               'LEG_08_16',
               'LEG_08_17',
               'LEG_09_13',
               'LEG_09_17',
               'LEG_10_13',
               'LEG_13_14',
               'LEG_14_14',
               'LEG_17_12',
               'LEG_17_21',
               'LEG_17_25',
               'LEG_18_24'],
 'pairing 10': ['LEG_02_5', 'LEG_03_7'],
 'pairing 100': ['LEG_01_6', 'LEG_01_8'],
 'pairing 101': ['LEG_31_15', 'LEG_31_17'],
 'pairing 102': ['LEG_08_31', 'LEG_08_0'],
 'pairing 103': ['LEG_26_12', 'LEG_26_23'],
 'pairing 104': ['LEG_10_33',
                 'LEG_11_10',
                 'LEG_11_11',
                 'LEG_11_19',
                 'LEG_11_23',
                 'LEG_12_19'],
 'pairing 105': ['LEG_13_28', 'LEG_13_2'],
 'pairing 106': ['LEG_28_2', 'LEG_28_30'],
 'pairing 107': ['LEG_26_24', 'LEG_26_25'],
 'pairing 108': ['LEG_03_33',
                 'LEG_04_10',
                 'LEG_07_17',
                 'LEG_08_13',
   

In [54]:
print('Key Performance Indicators:')
pprint(kpi)

Key Performance Indicators:
{'avg flying hr time': 7.59,
 'flying hours standard deviation ': 7.51,
 'max flying hr time': 28.58,
 'max rest time': 872.0,
 'min flying hr time': 1.3,
 'min rest time': 18.0,
 'number of deadhead flights': 1,
 'number of flight legs not covered': 0,
 'pairings': 248}


### 4. Clean-up

#### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

